In [1]:
from metatrader.utils.factory import MetatraderFactory
metatrader = MetatraderFactory.get_metatrader()
import pandas as pd
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

mt5 = metatrader.connect()

symbol = 'SP500'
timeframe = mt5.TIMEFRAME_M1

trade_volume = 10.0
commission = 0.05



Connected to Metatrader successfully.


In [2]:
bars = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, 0,5000))
bars['time'] = pd.to_datetime(bars['time'], unit='s')
bars['trade_volume'] = trade_volume
bars

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume
0,2023-10-20 03:38:00,4267.2,4267.2,4266.4,4266.4,18,2,0,10.0
1,2023-10-20 03:39:00,4266.7,4267.2,4266.4,4266.9,25,4,0,10.0
2,2023-10-20 03:40:00,4267.0,4267.2,4266.4,4266.7,25,3,0,10.0
3,2023-10-20 03:41:00,4266.6,4267.2,4266.6,4267.2,13,4,0,10.0
4,2023-10-20 03:42:00,4267.2,4267.9,4267.2,4267.9,12,3,0,10.0
...,...,...,...,...,...,...,...,...,...
4995,2023-10-25 18:22:00,4221.0,4221.1,4215.6,4216.1,166,1,0,10.0
4996,2023-10-25 18:23:00,4215.9,4217.6,4214.9,4216.2,170,2,0,10.0
4997,2023-10-25 18:24:00,4216.3,4216.9,4214.7,4215.3,177,1,0,10.0
4998,2023-10-25 18:25:00,4215.2,4215.4,4212.3,4213.4,162,2,0,10.0


In [3]:
bars['ema_20'] = bars['close'].ewm(span=20).mean()
bars['ema_10'] = bars['close'].ewm(span=10).mean()
bars['hour'] = bars['time'].dt.hour
bars

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume,ema_20,ema_10,hour
0,2023-10-20 03:38:00,4267.2,4267.2,4266.4,4266.4,18,2,0,10.0,4266.400000,4266.400000,3
1,2023-10-20 03:39:00,4266.7,4267.2,4266.4,4266.9,25,4,0,10.0,4266.662500,4266.675000,3
2,2023-10-20 03:40:00,4267.0,4267.2,4266.4,4266.7,25,3,0,10.0,4266.676270,4266.685050,3
3,2023-10-20 03:41:00,4266.6,4267.2,4266.6,4267.2,13,4,0,10.0,4266.827463,4266.854703,3
4,2023-10-20 03:42:00,4267.2,4267.9,4267.2,4267.9,12,3,0,10.0,4267.086900,4267.154779,3
...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2023-10-25 18:22:00,4221.0,4221.1,4215.6,4216.1,166,1,0,10.0,4220.297802,4220.628241,18
4996,2023-10-25 18:23:00,4215.9,4217.6,4214.9,4216.2,170,2,0,10.0,4219.907536,4219.823106,18
4997,2023-10-25 18:24:00,4216.3,4216.9,4214.7,4215.3,177,1,0,10.0,4219.468723,4219.000723,18
4998,2023-10-25 18:25:00,4215.2,4215.4,4212.3,4213.4,162,2,0,10.0,4218.890749,4217.982410,18


In [4]:
bars2 = bars.dropna().copy()

def get_signal(x):

    if 16 <= x['hour'] <= 20:

        if x['ema_10'] > x['ema_20']:
            x["position"] = "long"
            return 1
        elif x['ema_10'] < x['ema_20']:
            x["position"] = "short"
            return -1

    return 0

bars2['signal'] = bars2.apply(get_signal, axis=1)
bars2['prev_price'] = bars2['close'].shift(1)
bars2['price_change'] = bars2['close'] - bars2['prev_price']

bars2['signal_change'] = (bars2['signal'] - bars2['signal'].shift(1)).abs()
bars2['commission'] = bars2.apply(lambda x: commission * x['signal_change'] * x['trade_volume'], axis=1)
bars2

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume,ema_20,ema_10,hour,signal,prev_price,price_change,signal_change,commission
0,2023-10-20 03:38:00,4267.2,4267.2,4266.4,4266.4,18,2,0,10.0,4266.400000,4266.400000,3,0,NaN,NaN,NaN,NaN
1,2023-10-20 03:39:00,4266.7,4267.2,4266.4,4266.9,25,4,0,10.0,4266.662500,4266.675000,3,0,4266.4,0.5,0.0,0.0
2,2023-10-20 03:40:00,4267.0,4267.2,4266.4,4266.7,25,3,0,10.0,4266.676270,4266.685050,3,0,4266.9,-0.2,0.0,0.0
3,2023-10-20 03:41:00,4266.6,4267.2,4266.6,4267.2,13,4,0,10.0,4266.827463,4266.854703,3,0,4266.7,0.5,0.0,0.0
4,2023-10-20 03:42:00,4267.2,4267.9,4267.2,4267.9,12,3,0,10.0,4267.086900,4267.154779,3,0,4267.2,0.7,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,2023-10-25 18:22:00,4221.0,4221.1,4215.6,4216.1,166,1,0,10.0,4220.297802,4220.628241,18,1,4220.9,-4.8,0.0,0.0
4996,2023-10-25 18:23:00,4215.9,4217.6,4214.9,4216.2,170,2,0,10.0,4219.907536,4219.823106,18,-1,4216.1,0.1,2.0,1.0
4997,2023-10-25 18:24:00,4216.3,4216.9,4214.7,4215.3,177,1,0,10.0,4219.468723,4219.000723,18,-1,4216.2,-0.9,0.0,0.0
4998,2023-10-25 18:25:00,4215.2,4215.4,4212.3,4213.4,162,2,0,10.0,4218.890749,4217.982410,18,-1,4215.3,-1.9,0.0,0.0


In [5]:
bars3 = bars2.dropna().copy()
bars3 = bars3.loc[bars3['signal_change']!=0]
bars3['profit'] = bars3['signal'] * bars3['price_change'] * bars3['trade_volume']
bars3['gross_profit'] = bars3['profit'].cumsum()
bars3['net_profit'] = bars3['gross_profit'] - bars3['commission'].cumsum()


bars3

,time,open,high,low,close,tick_volume,spread,real_volume,trade_volume,ema_20,ema_10,hour,signal,prev_price,price_change,signal_change,commission,profit,gross_profit,net_profit
740,2023-10-20 16:00:00,4260.7,4262.9,4260.7,4261.9,89,1,0,10.0,4261.325705,4261.367614,16,1,4260.7,1.2,1.0,0.5,12.0,12.0,11.5
792,2023-10-20 16:52:00,4265.8,4267.7,4264.9,4265.4,277,1,0,10.0,4268.255278,4268.138013,16,-1,4265.7,-0.3,2.0,1.0,3.0,15.0,13.5
868,2023-10-20 18:08:00,4239.5,4240.1,4237.7,4238.4,148,1,0,10.0,4237.948728,4237.984825,18,1,4239.7,-1.3,2.0,1.0,-13.0,2.0,-0.5
870,2023-10-20 18:10:00,4238.6,4238.6,4234.3,4234.7,200,2,0,10.0,4237.686827,4237.464221,18,-1,4238.5,-3.8,2.0,1.0,38.0,40.0,36.5
901,2023-10-20 18:41:00,4232.9,4234.6,4232.7,4234.4,154,2,0,10.0,4232.268630,4232.349949,18,1,4232.9,1.5,2.0,1.0,15.0,55.0,50.5
928,2023-10-20 19:08:00,4237.3,4237.3,4234.9,4235.6,140,2,0,10.0,4238.616431,4238.441327,19,-1,4237.4,-1.8,2.0,1.0,18.0,73.0,67.5
941,2023-10-20 19:21:00,4238.1,4240.4,4237.3,4239.7,128,2,0,10.0,4236.749346,4236.819874,19,1,4238.0,1.7,2.0,1.0,17.0,90.0,83.5
986,2023-10-20 20:06:00,4244.6,4245.3,4242.3,4242.5,153,2,0,10.0,4246.651954,4246.520753,20,-1,4244.6,-2.1,2.0,1.0,21.0,111.0,103.5
997,2023-10-20 20:17:00,4250.1,4252.3,4247.4,4251.7,289,2,0,10.0,4245.280750,4245.714151,20,1,4250.2,1.5,2.0,1.0,15.0,126.0,117.5
1019,2023-10-20 20:39:00,4249.8,4250.3,4247.9,4248.2,63,3,0,10.0,4249.657564,4249.647668,20,-1,4249.8,-1.6,2.0,1.0,16.0,142.0,132.5


In [6]:
px.line(bars3, 'time', ['gross_profit', 'net_profit'], title="SP500 NY session Backtest EMA Trendfollowing")


c:\Users\praveenjaisankar\AppData\Local\pypoetry\Cache\virtualenvs\metatrader-Cdlomroj-py3.10\lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()
